### **Random Forest**

---

Random Forest es un **algoritmo de aprendizaje automático** de uso común, registrado por Leo Breiman y Adele Cutler, que combina el resultado de múltiples árboles de decisión para llegar a un resultado único. Su facilidad de uso y flexibilidad han impulsado su adopción, ya que maneja problemas de clasificación y regresión. [1]
\
\
Pero, ¿qué son los árboles de decisión? Son estructuras algorítmicas que se inician con una pregunta. A partir de ahí, pueden subdividirse en una secuencia de preguntas para determinar una respuesta de tipo binario.
\
\
Estas preguntas constituyen los nodos de decisión en el árbol, que funcionan como un medio para dividir los datos. Cada pregunta ayuda a determinar una decisión final denominada "nodo hoja" o "leaf node".
\
\
Con el objetivo de profundizar en el entendimiento de los conceptos matemáticos, estadísticos, probabilísticos, computacionales y de programación que subyacen al algoritmo de Random Forest, he asumido la enorme empresa de programarlo desde sus cimientos hasta su puesta en funcionamiento, para finalmente comparar su rendimiento con lo ofrecido por una de las librerías más populares de Machine Learning: Scikit-learn.
\
\
A continuación se presenta el roadmap de este proyecto:

```mermaid
flowchart LR;
    A[Algoritm Random Forest] --> B[Decision Tree] --> E["TreeNode() - Unidad básica"];
	B[Decision Tree] --> F["entropy(y) - Mide la impureza"];
	B[Decision Tree] --> G["find_best_split(x, y) - Busca la mejor división"];
	B[Decision Tree] --> H["split_data(X, y, feature, threshold) - Divide datos"];
	B[Decision Tree] --> I["build_tree(X, y) - Crea el árbol"];
	B[Decision Tree] --> J["predict_tree(tree, X) - Predicción del árbol"];
	B[Decision Tree] --> K["DecisionTreeClassifier - Interface"];
	A[Algoritm Random Forest] --> C[Random Forest Classifier];
	C[Random Forest Classifier] --> L["bootstrap_sample(X, y) - Genera subconjuntos bootstrap"];
	C[Random Forest Classifier] --> M["fit(X, y) - Entrena múltiples árboles"];
	C[Random Forest Classifier] --> N["predict(X) - Votación mayoritaria"];
	C[Random Forest Classifier] --> Ñ["feature_importances - Importancia de variables"];
	A[Algoritm Random Forest] --> D[Perfomance];
	D[Perfomance] --> O["accuracy_score(y_true, y_pred) - Calcula la precisión"];
	D[Perfomance] --> P["confusion_matrix(y_true, y_pred) - Matriz de confusión"];
	D[Perfomance] --> Q["predict(X) -Votación mayoritaria"];
	D[Perfomance] --> R["plot_feature_importance(rf_model) - Visualiza importancia de variables"];
```

\
\
[1] IBM, _"¿Qué es el Random Forest?"_ [Online]. Available: https://www.ibm.com/mx-es/topics/random-forest. [Accessed: 28-Feb-2025].


#### **TreeNode:** el átomo de los Árboles de Decisión.

---

Los Árboles de Decisión son un tipo de algoritmo de aprendizaje supervisado no paramétrico. Están compuestos por una estructura jerárquica de árbol, que consta de un nodo raíz, ramas, nodos internos y nodos hoja. [2]

Los nodos representan el punto de decisión en donde el algoritmo debe optar por un camino. El **nodo raíz** es el subconjunto de datos inicial. Los **nodos internos** de decisión son aquellos estadios intermedios que permiten ramificar aún más la toma de decisiones. Finalmente, cuando el algoritmo obtiene un valor de predicción y detiene la división del subconjunto de datos, se lo considera un **nodo hoja**.

**TreeNode** es la clase que abstrae las características de los nodos y nos permite instanciar los distintos tipos. A continuación, se describen sus parámetros:

-   **`feature_index`:** índice de la variable utilizada para dividir el nodo.
-   **`threshold`:** valor umbral para la división.
-   **`left`:** nodo hijo izquierdo.
-   **`right`:** nodo hijo derecho.
-   **`value`:** valor predictorio para los nodos hoja.

[2] IBM, "¿Qué es un árbol de decisión?" [Online]. Available: https://www.ibm.com/es-es/think/topics/decision-trees. [Accessed: 28-Feb-2025].


In [1]:
from IPython.display import Markdown, display

In [2]:
from tree_decision.TreeNode import TreeNode

In [3]:
leaf_node = TreeNode()

In [4]:
internal_node = TreeNode(feature_index=2, threshold=3.5, left=leaf_node, right=None)

In [5]:
print("Nodo hoja: ", leaf_node.__dict__)
print("Nodo interno: ", internal_node.__dict__)

Nodo hoja:  {'feature_index': None, 'threshold': None, 'left': None, 'right': None, 'value': None}
Nodo interno:  {'feature_index': 2, 'threshold': 3.5, 'left': <tree_decision.TreeNode.TreeNode object at 0x00000183B4CD8C20>, 'right': None, 'value': None}


### **Mecanismos de decisión**

---

La entropía es una métrica que nos permite calcular la impureza de un conjunto de datos, es decir, su grado de desorden o incertidumbre. Otra métrica muy utilizada en los algoritmos de Random Forest es la medida de Gini. Sin embargo, para este ejercicio práctico utilizaremos la entropía como mecanismo de decisión.
\
\
Para poder comprender en profundidad este concepto supongamos que debemos desarrollar un algoritmo de predicción que pueda detectar cuáles de los emails que nos llegan a nuestra bandeja son spam y cuáles no. Veamos los siguientes escenarios:

-   **Escenario 1:** 5 emails son spam y 5 emails no lo son.
-   **Escenario 2:** 7 emails son spam y 3 emails no lo son.
-   **Escenario 3:** 9 emails son spam y 1 email no lo es.
-   **Escenario 4:** Los 10 emails son spam.


In [6]:
import numpy as np
from tree_decision.entropy import entropy

In [7]:
# Emails spam -> 1
# Emails no spam -> 0
scenary_one = np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])
scenary_two = np.array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0])
scenary_three = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0])
scenary_four = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Dado los distintos escenarios vamos a calcular la entropía para cada uno de ellos. Su ecuación viene dada por:

$$
H(x) = - \sum_{x=0}^{n} p(x) \log_2(p(x))
$$


In [8]:
entropy_scenary_one = entropy(scenary_one)
entropy_scenary_two = entropy(scenary_two)
entropy_scenary_three = entropy(scenary_three)
entropy_scenary_four = entropy(scenary_four)

Ahora que tenemos los resultados de la entropía para los diferentes escenarios, podemos analizar qué nos dicen en términos de impureza e incertidumbre de los conjuntos de datos.

**Escenario 1:** 5 emails son spam y 5 emails no lo son.

-   **Entropía:** `1.0`  
    La entropía en este caso es 1.0, lo que indica que el conjunto de datos está completamente equilibrado entre las dos clases (spam y no spam). Esto significa que la incertidumbre es máxima, ya que no hay ninguna preferencia hacia una clase, lo que nos dice que el modelo necesita el máximo de información para hacer una predicción correcta.

**Escenario 2:** 7 emails son spam y 3 emails no lo son.

-   **Entropía:** `0.881`  
    En este escenario, la entropía es de 0.881, lo que es más bajo que el valor de 1.0 del escenario anterior. Aunque hay más emails spam que no spam, aún hay algo de incertidumbre porque no es una clasificación completamente homogénea. La impureza sigue siendo relativamente alta, pero la división de clases es algo más clara.

**Escenario 3:** 9 emails son spam y 1 email no lo es.

-   **Entropía:** `0.469`  
    Aquí la entropía es de 0.469, lo que indica que el conjunto es mucho más homogéneo que en los casos anteriores. La mayoría de los emails son spam, lo que reduce considerablemente la incertidumbre. Aunque no es completamente puro, la impureza ha disminuido, lo que indica que la división de clases es más clara.

**Escenario 4:** Los 10 emails son spam.

-   **Entropía:** `0.0`  
    Finalmente, en este escenario donde todos los emails son spam, la entropía es 0.0. Esto significa que el conjunto es completamente homogéneo, y no hay ninguna incertidumbre ni impureza. Todos los elementos pertenecen a la misma clase, por lo que el modelo no necesita ninguna información adicional para hacer una predicción.


In [9]:
print("Entropy scenary one: ", entropy_scenary_one)
print("Entropy scenary two: ", entropy_scenary_two)
print("Entropy scenary three: ", entropy_scenary_three)
print("Entropy scenary four: ", entropy_scenary_four)

Entropy scenary one:  1.0
Entropy scenary two:  0.8812908992306927
Entropy scenary three:  0.4689955935892812
Entropy scenary four:  -0.0


### **La forma más eficiente de dividir los datos**

---

Los Árboles de Decisión dividen y ramifican un conjunto de datos con el objetivo de llegar a una conclusión final. Pero, ¿cuál es la manera más eficiente y efectiva de realizar esta segmentación? ¿Bajo qué criterio se dividen los datos?

La función `find_best_split(X, y)` implementa un procedimiento para determinar el umbral óptimo de división dentro de un conjunto de datos.  
Recibe dos argumentos:

-   **X**: Matriz de características (variables independientes) del dataset.
-   **y**: Vector objetivo que contiene las etiquetas o valores a predecir.

Este método sigue **ocho pasos fundamentales**, que se detallan a continuación:

1. **Inicializa un diccionario `results`**, que almacenará los umbrales evaluados junto con la entropía total correspondiente.
2. **Recorre cada una de las características (columnas) de X**, iterando sobre todas las variables disponibles.
3. **Ordena los valores de cada característica de menor a mayor** para evaluar posibles puntos de corte.
4. **Calcula los valores intermedios entre cada par consecutivo de datos ordenados** y los utiliza iterativamente como posibles _thresholds_ (umbrales) de división.
5. **Divide el conjunto de datos** en dos subconjuntos:
    - **`left_y`**: Conjunto de valores objetivo asociados a observaciones menores o iguales al umbral.
    - **`right_y`**: Conjunto de valores objetivo asociados a observaciones mayores al umbral.
6. **Calcula la entropía de cada subconjunto**, una medida del grado de desorden o impureza en los datos resultantes de la división.
7. **Calcula la entropía total**, ponderando la entropía de cada subconjunto en función de la cantidad de elementos que contiene respecto al total de datos en `X`. Luego, almacena en `results` el umbral evaluado junto con su respectiva entropía total.
8. **Devuelve el valor del umbral (threshold) que minimiza la entropía total**, es decir, aquel que genera la división más homogénea posible dentro del conjunto de datos.

Este proceso garantiza que la segmentación de los datos sea lo más informativa posible, favoreciendo la creación de nodos homogéneos dentro del Árbol de Decisión.


In [10]:
from tree_decision.find_best_split import find_best_split

### **Ejemplo de división de datos con `find_best_split`**

---

Para este ejemplo, tenemos dos arreglos:

-   **`X`**: Una matriz que representa las características o variables independientes del conjunto de datos, el cual agrupa emails clasificados como **spam (1)** o **no spam (0)**.
-   **`y`**: Un vector que contiene los valores objetivo (_target_), indicando si un email es spam (`1`) o no (`0`).

Cada columna de la matriz `X` representa una característica específica del contenido del email:

1. **Primera columna**: Cantidad de veces que aparece la frase `"Sólo por hoy"`.
2. **Segunda columna**: Cantidad de signos de exclamación (`!`).
3. **Tercera columna**: Cantidad de veces que aparece la palabra `"Oferta"`.

El conjunto de datos es el siguiente:


In [11]:
X = np.array([[1, 5, 1], [3, 2, 2], [4, 6, 1], [6, 8, 3], [1, 5, 4]])
y = np.array([1, 0, 1, 0, 1])

Al ejecutar la función `find_best_split`, obtenemos una tupla con tres valores:

**Índice de la mejor característica (feature):** Indica qué columna de X proporciona la mejor división.  
**Umbral óptimo (threshold):** Punto de corte que genera la separación más homogénea del conjunto de datos.  
**Entropía total:** Medida de la impureza resultante tras la división de los datos.


In [12]:
feature, threshold, entropy = find_best_split(X, y)

In [13]:
print("feature (índice): ", feature)
print("threshold (umbral de corte): ", threshold)
print("entropy (entropía ponderada): ", entropy)

feature (índice):  0
threshold (umbral de corte):  1.0
entropy (entropía ponderada):  0.5509775004326937
